In [1]:
from model_coupled import CoupledBert
from classifier import Classifier
from nli import NLIModel
from transformers import BertModel, AutoTokenizer
from utils import load_model
import numpy as np
import pandas as pd
import pathlib
from tqdm import tqdm
import torch



cbert_tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
cbert_tokenizer.add_tokens(['[QUERY]','[DESCRIPTION]','[NUM]'])
cbert = BertModel.from_pretrained("sbert107")
cbert.resize_token_embeddings(cbert.embeddings.word_embeddings.weight.shape[0]+3)
#В данном случае классификатор лишь забирает ресурсы
# Его смысл в том, сужать диапазон классов, и оставлять только top-k для последующей зеро-шот классификации, 
# регулируя в параметр k можно избавить модель от некоторого количества ошибок первого рода, что может увеличить качество при правильном обучении
# Но в последнем чек-поинте модель лучше справляется с максимальным топ-k = N классов, тем не менее является частью решения, и возможно выигрышным был сабмит с топ_к = 9 и с весами, которые не сохранились
cmodel = Classifier(cbert,cbert_tokenizer,mask_prob=0)
cmodel.eval()

load_model(cmodel,"gg_/checkpoint-1200")
cmodel.requires_grad_(False)



bert_tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
bert_tokenizer.add_tokens(['[QUERY]','[DESCRIPTION]','[NUM]'])
bert = BertModel.from_pretrained("10-8nli")
bert.eval()
top_k = 50
model = NLIModel(bert,bert_tokenizer,embedding_dim=512,k=top_k,mask_prob=0)
# Не могу быть уверенным, что именно этот чекпоинт дал результат в приватном либердорте, но это последний топ-1 на паблике, если выиграл тот, который был 2 дня назад, то, он должен быть в одной из тех папочек, только возможно придется поменять количество и размер адаптеров, чтобы загружались без предупреждений, бест трешолды и топ-к можно вычислить в ноутбуке evaluation, там всё для этого захардкодено
load_model(model,"10-8mean_sbert8___/checkpoint-404")
bert.requires_grad_(False)



model = CoupledBert(model,bert_tokenizer,top_k,50,cmodel,cbert_tokenizer).cuda()
model.eval()

model.prepare_description_tokens()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


trainable params: 131,716,096 || all params: 434,025,472 || trainable%: 30.3475


c:\dls-workshop\utils.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p = torch.load(f"{cp}/{name}").requires_grad_(False)


trainable params: 589,824 || all params: 427,501,568 || trainable%: 0.1380


c:\dls-workshop\utils.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p = torch.load(f"{cp}/{name}").requires_grad_(False)
Asking to truncate to max_length but no maximu

In [2]:
from data import train_dataset,test_dataset

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
Map: 100%|██████████| 22188/22188 [00:00<00:00, 30810.57 examples/s]


In [3]:
ROOT_DIR = pathlib.Path().absolute()
DATA_DIR = ROOT_DIR / "data"
RANDOM_SEED = 42
test = pd.read_csv(DATA_DIR / "test.csv")

In [4]:
model.eval()
result = []
with torch.no_grad():
    for i in tqdm(test['text']):
        
        if isinstance(i, str):
            i = '[QUERY]'+i
            outputs,desc,classes,_ = model.predict(i,k=top_k)
        else:
            outputs,desc,classes,_ = model.predict('[QUERY].',k=top_k)
        
        desc = desc.squeeze(1)

        similarities = torch.nn.functional.cosine_similarity(outputs,desc)
        preds = classes[torch.where(similarities > 0.819)].sort().values.detach().cpu().numpy().astype(int).astype(str).tolist()

        if len(preds)==0:
            preds = [classes[torch.argmax(similarities)].sort().values.detach().cpu().numpy().astype(int).astype(str).tolist()]
            
        result.append(' '.join(preds))

100%|██████████| 9015/9015 [05:22<00:00, 27.98it/s]


In [6]:
res = pd.DataFrame(np.hstack([test["index"].values.reshape(-1,1), np.array(result).reshape(-1,1)]),
                  columns = ["index","target"])
res.to_csv(DATA_DIR / "submission.csv", index=False)